### Stable Diffusion

***x1101's Stable Diffusion 1.5 Webui***



#### Installation

In [ ]:
# Run this cell to install the WebUI or just Update it along with all Extensions installed. 

%cd /home/studio-lab-user
print("Starting...")
root_path = "/home/studio-lab-user/content"
webui_path = "/home/studio-lab-user/content/x1101"

!pip install -q --upgrade torchsde
!pip install -q --upgrade pip
!pip install -q --upgrade psutil
!pip install -q git+https://github.com/DEX-1101/colablib
!pip install -q pytz
import os
import time
from colablib.colored_print import cprint, print_line
from colablib.utils import py_utils
from IPython.display import clear_output
start_time    = time.time()

print_line(0, color="green")
cprint("[+] Preparing Environment...", color="flat_cyan")
%conda update -q -n base conda
%conda install -q -c conda-forge gputil
%conda install -q -y aria2
%conda install -q -y glib
%pip install -q opencv-python-headless huggingface-hub
!pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.20 triton==2.0.0 -U

if not os.path.exists(webui_path):
    print_line(0, color="green")
    cprint("[+] Downloading Repository...", color="flat_cyan")
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/x1101/fix/resolve/main/zip/fastrepo_ext_def.zip -o fastrepo_ext.zip
    cprint("Unpacking...", color="yellow")
    !unzip -q -o fastrepo_ext.zip 
    !mv /home/studio-lab-user/home/studio-lab-user/content /home/studio-lab-user
    !git clone -q https://github.com/zanllp/sd-webui-infinite-image-browsing {webui_path}/extensions/sd-webui-infinite-image-browsing
    !rm fastrepo_ext.zip
    !rm -rf /home/studio-lab-user/home
    !wget -q https://raw.githubusercontent.com/DEX-1101/SecretNAI/main/template.txt -O {root_path}/download_list.txt
else:
    print_line(0, color="green")
    cprint("[+] Repository already exists. Skipping...", color="flat_cyan")  

print_line(0, color="green")
cprint("[+] Updating Repository...", color="flat_cyan")
%cd {webui_path}
!git pull
print_line(0, color="green")
cprint("[+] Updating Extensions...", color="flat_cyan")
%cd {webui_path}/extensions
!find . -mindepth 1 -maxdepth 1 -type d -print -exec git -C {} pull \;

clear_output()
cprint("WebUI Updated !", color="flat_yellow")
%cd {webui_path}
!git reset --hard
print_line(0, color="green")
cprint("Extensions Updated !", color="flat_yellow")
%cd {webui_path}/extensions
!find . -mindepth 1 -maxdepth 1 -type d -print -exec git -C {} reset --hard \;
elapsed_time  = py_utils.calculate_elapsed_time(start_time)
print_line(0, color="green")
cprint(f"[+] All completed within {elapsed_time}.", color="flat_cyan")

#### Download Section

In [ ]:
# Example download using Pastebin : https://pastebin.com/ZPfcNx3u
pastebin_url = ""   # Download Model, LoRA, VAE, Embeddings, or Extension from Pastebin URL. Leave it empty if using "download_list.txt".
hf_token = ""       # Put Hugging face token here to download from private repository. Leave empty if NOT.
# Open the "download_list.txt" file inside "/content" folder and input link there on the corresponding #hastag if you're not using Pastebin.

#=================================================

#Code by Bang Furqanil
import os
import time
from colablib.utils import py_utils
from pydantic import BaseModel
from colablib.utils.py_utils import get_filename
from colablib.sd_models.downloader import aria2_download, download
from colablib.colored_print import cprint, print_line
from colablib.utils.config_utils import read_config
from colablib.utils.git_utils import clone_repo

%store -r
root_dir            = "/home/studio-lab-user/content"
repo_dir            = os.path.join(root_dir, "x1101")
custom_model_url        = ""
custom_vae_url          = ""
custom_embedding_url    = ""
custom_LoRA_url         = ""
custom_extensions_url   = ""
models_dir          = os.path.join(repo_dir, "models", "Stable-diffusion")
vaes_dir            = os.path.join(repo_dir, "models", "VAE")
lora_dir            = os.path.join(repo_dir, "models", "Lora")
embeddings_dir      = os.path.join(repo_dir, "embeddings")
extensions_dir      = os.path.join(repo_dir, "extensions")
download_list       = os.path.join(root_dir, "download_list.txt")
class CustomDirs(BaseModel):
    url: str
    dst: str
user_header = f"Authorization: Bearer {hf_token}"
custom_dirs = {
    "model"       : CustomDirs(url=custom_model_url, dst=models_dir),
    "vae"         : CustomDirs(url=custom_vae_url, dst=vaes_dir),
    "embedding"   : CustomDirs(url=custom_embedding_url, dst=embeddings_dir),
    "lora"        : CustomDirs(url=custom_LoRA_url, dst=lora_dir),
    "extensions"  : CustomDirs(url=custom_extensions_url, dst=extensions_dir),
}

def parse_urls(filename):
    content = read_config(filename)
    lines   = content.strip().split('\n')
    result  = {}
    key     = ''
    for line in lines:
        if not line.strip():
            continue
        if line.startswith('//'):
            continue
        if line.startswith('#'):
            key = line[1:].lower()
            result[key] = []
        else:
            urls = [url.strip() for url in line.split(',') if url.strip() != '']
            result[key].extend(urls)
    return result

def get_filename(url, token=None):
    headers = {}
    if token:
        headers['Authorization'] = f'Bearer {hf_token}'

def custom_download(custom_dirs):
    for key, value in custom_dirs.items():
        urls     = value.url.split(",")  # Split the comma-separated URLs
        dst      = value.dst

        if value.url:
            print_line(0, color="green")
            cprint(f" [+] Downloading {key}.", color="flat_yellow")

        for url in urls:
            url = url.strip()  # Remove leading/trailing whitespaces from each URL
            if url != "":
                print_line(0, color="green")
                if "|" in url:
                    url, filename = map(str.strip, url.split("|"))
                    if not filename.endswith((".safetensors", ".ckpt", ".pt", "pth")):
                        filename = filename + os.path.splitext(get_filename(url))[1]
                else:
                    if not url.startswith("fuse:"):
                        filename = get_filename(url)

                if url.startswith("fuse:"):
                    fuse(url, key, dst)
                elif key == "extensions":
                    clone_repo(url, cwd=dst)
                else:
                   download(url=url, filename=filename, user_header=user_header, dst=dst, quiet=False)

def download_from_textfile(filename):
    for key, urls in parse_urls(filename).items():
        key_lower = key.lower()
        if key_lower in custom_dirs:
            if custom_dirs[key_lower].url:
                custom_dirs[key_lower].url += ',' + ','.join(urls)
            else:
                custom_dirs[key_lower].url = ','.join(urls)
        else:
            cprint(f"Warning: Category '{key}' from the file is not found in custom_dirs.", color="flat_yellow")

def custom_download_list(url):
    filename = "custom_download_list.txt"
    filepath = os.path.join(root_dir, filename)
    if os.path.exists(filepath):
        os.remove(filepath)
    if 'pastebin.com' in url:
        if 'raw' not in url:
            url = url.replace('pastebin.com', 'pastebin.com/raw')
    download(url=url, filename=filename, user_header=user_header, dst=root_dir, quiet=True)
    return filepath

def main():
    start_time    = time.time()
    textfile_path = download_list
    if pastebin_url:
        textfile_path = custom_download_list(pastebin_url)
    download_from_textfile(textfile_path)
    custom_download(custom_dirs)

    elapsed_time  = py_utils.calculate_elapsed_time(start_time)
    print_line(0, color="green")
    cprint(f"[+] Download completed within {elapsed_time}.", color="flat_yellow")
main()

In [ ]:
# Set to "1" to download the model.

canny = "1"
openpose = "0"
depth = "1"
normal_map = "0"
mlsd = "0"
lineart = "0"
soft_edge = "0"
scribble = "0"
segmentation = "0"
shuffle = "0"
tile = "0"
inpaint = "0"
instruct_p2p = "0"
t2i_adapter = "0"

cnet_dir = "/home/studio-lab-user/content/x1101/extensions/sd-webui-controlnet/models"

#=================================================

from colablib.colored_print import cprint, print_line
print_line(0, color="green")
cprint(f"[+] Downloading...", color="flat_cyan")
%cd {cnet_dir}
def main():
    if canny == "1":
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors -o control_v11p_sd15_canny_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_canny_fp16.yaml -o control_v11p_sd15_canny_fp16.yaml
    if openpose == "1":
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -o control_v11p_sd15_openpose_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_openpose_fp16.yaml -o control_v11p_sd15_openpose_fp16.yaml
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.safetensors -o control_v1p_sd15_qrcode_monster.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/raw/main/control_v1p_sd15_qrcode_monster.yaml -o control_v1p_sd15_qrcode_monster.yaml
    if depth == "1":
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -o control_v11f1p_sd15_depth_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1p_sd15_depth_fp16.yaml -o control_v11f1p_sd15_depth_fp16.yaml
    if normal_map == "1":
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -o control_v11p_sd15_normalbae_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_normalbae_fp16.yaml -o control_v11p_sd15_normalbae_fp16.yaml
    if mlsd == "1":
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -o control_v11p_sd15_mlsd_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_mlsd_fp16.yaml -o control_v11p_sd15_mlsd_fp16.yaml
    if lineart == "1":
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -o control_v11p_sd15_lineart_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -o control_v11p_sd15s2_lineart_anime_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_lineart_fp16.yaml -o control_v11p_sd15_lineart_fp16.yaml
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15s2_lineart_anime_fp16.yaml -o control_v11p_sd15s2_lineart_anime_fp16.yaml
    if soft_edge == "1":
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -o control_v11p_sd15_softedge_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_softedge_fp16.yaml -o control_v11p_sd15_softedge_fp16.yaml
    if scribble == "1":
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -o control_v11p_sd15_scribble_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_scribble_fp16.yaml -o control_v11p_sd15_scribble_fp16.yaml
    if segmentation == "1":
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg_fp16.safetensors -o control_v11p_sd15_seg_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_seg_fp16.yaml -o control_v11p_sd15_seg_fp16.yaml
    if shuffle == "1":
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -o control_v11e_sd15_shuffle_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_shuffle_fp16.yaml -o control_v11e_sd15_shuffle_fp16.yaml
    if tile == "1":
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -o control_v11f1e_sd15_tile_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile_fp16.yaml -o control_v11f1e_sd15_tile_fp16.yaml
    if inpaint == "1": 
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -o control_v11p_sd15_inpaint_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_inpaint_fp16.yaml -o control_v11p_sd15_inpaint_fp16.yaml
    if instruct_p2p == "1":
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -o control_v11e_sd15_ip2p_fp16.safetensors
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_ip2p_fp16.yaml -o control_v11e_sd15_ip2p_fp16.yaml
    if t2i_adapter == "1":
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_style_sd14v1.pth -o t2iadapter_style_sd14v1.pth
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_sketch_sd14v1.pth -o t2iadapter_sketch_sd14v1.pth
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_seg_sd14v1.pth -o t2iadapter_seg_sd14v1.pth
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_openpose_sd14v1.pth -o t2iadapter_openpose_sd14v1.pth
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_keypose_sd14v1.pth -o t2iadapter_keypose_sd14v1.pth
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_depth_sd14v1.pth -o t2iadapter_depth_sd14v1.pth
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_color_sd14v1.pth -o t2iadapter_color_sd14v1.pth
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_canny_sd14v1.pth -o t2iadapter_canny_sd14v1.pth
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_canny_sd15v2.pth -o t2iadapter_canny_sd15v2.pth
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_depth_sd15v2.pth -o t2iadapter_depth_sd15v2.pth
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_sketch_sd15v2.pth -o t2iadapter_sketch_sd15v2.pth
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_zoedepth_sd15v1.pth -o t2iadapter_zoedepth_sd15v1.pth     
main()
print_line(0, color="green")
cprint(f"[+] All Download Completed.", color="flat_cyan")

#### Start Webui

In [ ]:
ngrok_token = "ur_token_here"   # Get your ngrok_oken here : https://dashboard.ngrok.com/get-started/your-authtoken

#=================================================

from colablib.colored_print import cprint, print_line
from IPython.display import clear_output
import GPUtil
import subprocess
import time
try:
    start_colab
except:
    start_colab = int(time.time())-5
def gpu_available():
    try:
        GPUs = GPUtil.getGPUs()
        return len(GPUs) > 0
    except Exception as e:
        return False
    
!echo -n {start_colab} > /home/studio-lab-user/content/x1101/static/colabTimer.txt
%cd /home/studio-lab-user/content/x1101
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /home/studio-lab-user/content/x1101/modules/shared_options.py
if gpu_available():
    cprint(f"[+] Running with GPU...", color="flat_cyan")
    !python launch.py --listen --xformers --theme dark --enable-insecure-extension-access --disable-console-progressbars --no-half-vae --ngrok {ngrok_token}
else:
    cprint(f"[+] Running with CPU...", color="flat_cyan")
    !python launch.py --skip-torch-cuda-test --theme dark --precision full --ngrok {ngrok_token} --no-half --use-cpu SD GFPGAN BSRGAN ESRGAN SCUNet CodeFormer --all

### Real-time Voice Changer

***w-okada's Voice Changer***

You can use the following settings for better results: \
`If you're using a index: f0: RMVPE_ONNX | Chunk: 112 or higher | Extra: 8192` \
`If you're not using a index: f0: RMVPE_ONNX | Chunk: 96 or higher | Extra: 16384`

Credits \
Realtime Voice Changer by [w-okada](https://github.com/w-okada/voice-changer)\
Notebook files updated by [rafacasari](https://github.com/Rafacasari)\
Recommended settings by [YunaOneeChan](https://github.com/YunaOneeChan)\
Updated to Sagemaker Studio Lab by [DaUnknow1](https://github.com/DaUnknow1)

#### Installation

In [ ]:
%cd /home/studio-lab-user/

!pip install colorama --quiet
from colorama import Fore, Style
import os

!git clone https://github.com/w-okada/voice-changer.git --quiet
print(f"{Fore.GREEN}> Successfully cloned the repository!{Style.RESET_ALL}")
%cd voice-changer/server
print(f"{Fore.CYAN}> Installing Portaudio...{Style.RESET_ALL}")
!conda install -y -c conda-forge portaudio --quiet #replacement for the libportaudio2 package
print(f"{Fore.CYAN}> Installing pre-dependencies...{Style.RESET_ALL}")
!pip install pyworld --no-build-isolation --quiet
!pip install fairseq pyngrok faiss-gpu jedi --quiet
print(f"{Fore.CYAN}> Installing dependencies from requirements.txt...{Style.RESET_ALL}")
!pip install -r requirements.txt --quiet
print(f"{Fore.GREEN}> Successfully installed all packages!{Style.RESET_ALL}")
#settings
!wget -q https://gist.githubusercontent.com/Rafacasari/d820d945497a01112e1a9ba331cbad4f/raw/8e0a426c22688b05dd9c541648bceab27e422dd6/kaggle_setting.json -O [PATH]
print("> You can safely ignore the DEPRECATION errors if there is any, it's doesn't interfer on Voice Changer!")

#### Start Webui

In [ ]:
Token = '2TB6cS53aTS5OtrEdEx6vzAmfEc_3Tbk4JsNYZCzfdAXPz69e' 
Region = "us" #ap - Asia/Pacific (Singapore)", "au - Australia (Sydney)","eu - Europe (Frankfurt)", "in - India (Mumbai)","jp - Japan (Tokyo)","sa - South America (Sao Paulo)", "us - United States (Ohio)"]


%cd /home/studio-lab-user/voice-changer/server


from pyngrok import conf, ngrok
MyConfig = conf.PyngrokConfig()
MyConfig.auth_token = Token
MyConfig.region = Region[0:2]
#conf.get_default().authtoken = Token
#conf.get_default().region = Region
conf.set_default(MyConfig);

import subprocess, threading, time, socket, urllib.request
PORT = 8000

from pyngrok import ngrok
ngrokConnection = ngrok.connect(PORT)
public_url = ngrokConnection.public_url

from IPython.display import clear_output

def wait_for_server():
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', PORT))
        if result == 0:
            break
        sock.close()
    print("--------- SERVER READY! ---------")
    print("Your server is available at:")
    print(public_url)
    print("---------------------------------")

threading.Thread(target=wait_for_server, daemon=True).start()

!python3 MMVCServerSIO.py \
  -p {PORT} \
  --https False \
  --content_vec_500 pretrain/checkpoint_best_legacy_500.pt \
  --content_vec_500_onnx pretrain/content_vec_500.onnx \
  --content_vec_500_onnx_on true \
  --hubert_base pretrain/hubert_base.pt \
  --hubert_base_jp pretrain/rinna_hubert_base_jp.pt \
  --hubert_soft pretrain/hubert/hubert-soft-0d54a1f4.pt \
  --nsf_hifigan pretrain/nsf_hifigan/model \
  --crepe_onnx_full pretrain/crepe_onnx_full.onnx \
  --crepe_onnx_tiny pretrain/crepe_onnx_tiny.onnx \
  --rmvpe pretrain/rmvpe.pt \
  --model_dir model_dir \
  --samples samples.json

ngrok.disconnect(ngrokConnection.public_url)

### Useful Command

#### Disk/Storage Information

In [ ]:
import os
from colablib.colored_print import cprint, print_line
import psutil
directory = "/home/studio-lab-user" # Change the path to check the others.
disk_space = psutil.disk_usage(directory)
total = disk_space.total / (1024 ** 3)
used = disk_space.used / (1024 ** 3)
free = disk_space.free / (1024 ** 3)
cprint(f"Total Storage on '{directory}' : [{total:.2f} GB] | Used: [{used:.2f} GB] | Free: [{free:.2f} GB]", color="flat_cyan")

#### Delete Section

***Delete all Lora, Model, Output, ControlNet Model and Extensions.***

In [ ]:
# Set to "1" to delete it.

delete_models             = "0"
delete_loras              = "0"
delete_controlnet_models  = "0"
delete_outputs            = "0"
delete_extensions         = "0"
delete_vae                = "0"

webui_path = "/home/studio-lab-user/content/x1101"

#=================================================

def main():
  if delete_loras == "1":
    !rm -f -d -r {webui_path}/models/Lora/*
  if delete_models == "1":
    !rm -f -d -r {webui_path}/models/Stable-diffusion/*
  if delete_controlnet_models == "1":
    !rm -f -d -r {webui_path}/extensions/sd-webui-controlnet/models/*
  if delete_outputs == "1":
    !rm -f -d -r {webui_path}/outputs/*
  if delete_extensions == "1":
    !rm -f -d -r {webui_path}/extensions/*
  if delete_vae == "1":
    !rm -f -d -r {webui_path}/models/VAE/*
main()
print("Done !")

#### Clean Unnecessary Cache and Files

In [ ]:
!pip cache purge
!rm -rf ~/.cache

#### Delete All Libraries or Entire Notebook

***Don't run this if you're don't know what are you doing.***

In [ ]:
%cd ~/
!pip freeze > reqs.txt
!pip uninstall -y -r reqs.txt

In [ ]:
!rm -rf /home/studio-lab-user/